<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# The Main Data Loading Pipeline Summarized

전체 챕터 코드는 ch02.ipynb

이 노트북은 중간 단계를 제외한 데이터 로딩 파이프라인의 핵심 요점만 포함함.

이 노트북에서 사용되는 패키지들:

라이브러리 버전 확인

torch와 tiktoken의 설치 버전을 확인,
호환성을 위해 현재 환경의 버전을 출력.

In [1]:
# NBVAL_SKIP
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.4.0
tiktoken version: 0.7.0


데이터셋 및 데이터 로더 클래스 구현

GPTDatasetV1 클래스:

__init__: 텍스트 전체를 토큰화한 뒤, sliding window 방식을 사용하여 max_length 길이만큼 입력을 자르고, 그 다음 토큰을 타겟으로 설정하여 input_ids와 target_ids를 생성함.

__getitem__: 인덱스에 해당하는 입력과 타겟 텐서를 반환함.

create_dataloader_v1 함수:

tiktoken의 GPT-2 토크나이저를 초기화하고, 위에서 만든 GPTDatasetV1을 이용해 PyTorch의 DataLoader를 생성하여 반환함. batch_size, shuffle 등의 옵션을 설정 가능.

the-verdict.txt 파일을 읽어와서 배치 크기 8, 문맥 길이 4인 데이터 로더를 생성.

In [2]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size, max_length, stride,
                         shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader


with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

vocab_size = 50257
output_dim = 256
context_length = 1024


token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

batch_size = 8
max_length = 4
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=batch_size,
    max_length=max_length,
    stride=max_length
)

임베딩 레이어 테스트

데이터 로더에서 첫 번째 배치를 가져와서 (x, y).

토큰 임베딩: 입력 토큰 x를 256차원 벡터로 변환.

위치 임베딩: 0부터 max_length까지의 위치 정보를 256차원 벡터로 변환.

입력 임베딩: 토큰 임베딩과 위치 임베딩을 더해서 최종적으로 모델에 들어갈 입력.

In [3]:
for batch in dataloader:
    x, y = batch

    token_embeddings = token_embedding_layer(x)
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))

    input_embeddings = token_embeddings + pos_embeddings

    break

결과 확인

배치 크기(8) x 문맥 길이(4) x 임베딩 차원(256)의 형태를 가진 텐서가 성공적으로 생성되었음을 확인.

In [4]:
print(input_embeddings.shape)

torch.Size([8, 4, 256])
